In [1]:
#import all libraries
import pandas as pd
import numpy as np

In [3]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Sequential
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional,LSTM, Dropout
from keras.layers.embeddings import Embedding
from tensorflow.keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy
from keras.callbacks import ModelCheckpoint

The dataset being used is available on https://www.kaggle.com/datasets/devicharith/language-translation-englishfrench.

In [4]:
df  =pd.read_csv('/kaggle/input/language-translation-englishfrench/eng_-french.csv')
df.head()

,English words/sentences,French words/sentences
0,Hi.,Salut!
1,Run!,Cours !
2,Run!,Courez !
3,Who?,Qui ?
4,Wow!,Ça alors !


Separating the English and French data.

In [5]:
eng = df['English words/sentences']
fr = df['French words/sentences']

Some interesting stats about the datasets.

In [6]:
import collections

english_words_counter = collections.Counter([word for sentence in eng for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in fr for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in eng for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in fr for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

1082098 English words.
27393 unique English words.
10 Most common words in the English dataset:
"I" "to" "you" "the" "a" "is" "Tom" "of" "in" "have"

1177832 French words.
44918 unique French words.
10 Most common words in the French dataset:
"de" "Je" "?" "pas" "que" "à" "ne" "la" "le" "Il"


In [7]:
def tokenize(x):
    
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(x)
    return tokenizer.texts_to_sequences(x), tokenizer


In [8]:
def pad(x, length=None):
    if length is None:
        length = max([len(sentence) for sentence in x])
    return pad_sequences(x, maxlen = 55, padding = 'post')
    

In [9]:
def preprocess(x, y):
   
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

In [10]:
preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer = preprocess(eng, fr)

In [11]:
max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

Max English sentence length: 55
Max French sentence length: 55
English vocabulary size: 14531
French vocabulary size: 30660


In [12]:
def logits_to_text(logits, tokenizer):
    
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

In [13]:
def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    
    learning_rate = 0.003
    
    # Build the layers
    model = Sequential()
    model.add(Embedding(french_vocab_size, 256, input_length=input_shape[1], input_shape=input_shape[1:]))
    model.add(Bidirectional(GRU(256, return_sequences=True)))
    model.add(TimeDistributed(Dense(1024, activation='relu')))
    model.add(Dropout(0.5))
    model.add(TimeDistributed(Dense(english_vocab_size, activation='softmax'))) 

    # Compile model
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model


In [16]:
# Reshape the input
tmp_x = pad(preproc_french_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))

# Train 
model = bd_model(
    tmp_x.shape,
    preproc_english_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(french_tokenizer.word_index)+1)

model.summary()

model.fit(tmp_x, preproc_english_sentences, batch_size=64, epochs=5, validation_split=0.2)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 55, 256)           7849216   
_________________________________________________________________
bidirectional (Bidirectional (None, 55, 512)           789504    
_________________________________________________________________
time_distributed (TimeDistri (None, 55, 1024)          525312    
_________________________________________________________________
dropout (Dropout)            (None, 55, 1024)          0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 55, 14532)         14895300  
Total params: 24,059,332
Trainable params: 24,059,332
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5
2196/2196 [==============================] - 257s 115ms/step - loss: 0.4980 - accuracy: 0.9274 - val

In [14]:
tmp_x.shape

NameError: name 'tmp_x' is not defined

In [15]:
preproc_english_sentences.shape

(175621, 55)

In [19]:
  i=100077


  print("Prediction:")
  print(logits_to_text(model.predict(tmp_x[[i]])[0], english_tokenizer))

  print("\nCorrect Translation:")
  print(eng[i])

  print("\nOriginal text:")
  print(fr[i])

SyntaxError: invalid syntax (1289797549.py, line 7)

In [20]:
model.save("translation_model.h5")
